# Conteo de q-Coloraciones en Lattices K×K

Método telescópico con MCMC (Gibbs sampler)

## Librerías

In [37]:
import numpy as np
from numba import jit
import pandas as pd
import time
import os

np.random.seed(42)
os.makedirs('../results', exist_ok=True)

## Funciones

In [38]:
# ============================================================================
# FUNCIONES BÁSICAS
# ============================================================================

def create_lattice_edges(K):
    """Crea aristas de lattice K×K con bordes libres."""
    edges = []
    # Horizontales: (x,y) → (x+1,y)
    for y in range(K):
        for x in range(K - 1):
            edges.append((x, y, x+1, y))
    # Verticales: (x,y) → (x,y+1)
    for y in range(K - 1):
        for x in range(K):
            edges.append((x, y, x, y+1))
    return np.array(edges, dtype=np.int64)


@jit(nopython=True)
def is_valid_coloring(coloring, edges):
    """Verifica si coloración es válida para grafo con aristas 'edges'."""
    for i in range(len(edges)):
        x1, y1, x2, y2 = edges[i]
        if coloring[x1, y1] == coloring[x2, y2]:
            return False
    return True


@jit(nopython=True)
def get_neighbor_colors(x, y, coloring, edges):
    """Obtiene colores de vecinos de (x,y) según aristas en 'edges'."""
    neighbor_colors = set()
    for i in range(len(edges)):
        x1, y1, x2, y2 = edges[i]
        if x1 == x and y1 == y:
            neighbor_colors.add(coloring[x2, y2])
        elif x2 == x and y2 == y:
            neighbor_colors.add(coloring[x1, y1])
    return neighbor_colors


@jit(nopython=True)
def gibbs_step_partial(coloring, edges, q):
    """
    Un paso del Gibbs sampler para grafo parcial.
    Muestrea sobre TODA la lattice K×K pero solo valida aristas en 'edges'.
    """
    K = coloring.shape[0]
    x = np.random.randint(0, K)
    y = np.random.randint(0, K)
    neighbor_colors = get_neighbor_colors(x, y, coloring, edges)
    valid_colors = []
    for c in range(q):
        if c not in neighbor_colors:
            valid_colors.append(c)
    if len(valid_colors) > 0:
        idx = np.random.randint(0, len(valid_colors))
        coloring[x, y] = valid_colors[idx]


@jit(nopython=True)
def run_gibbs_sampler_partial(coloring, edges, q, n_steps):
    """Ejecuta n_steps del Gibbs sampler."""
    for _ in range(n_steps):
        gibbs_step_partial(coloring, edges, q)


def estimate_ratio(K, edges_i_minus_1, edges_i, q, n_samples, n_steps_per_sample, max_steps):
    """
    Estima r_i = Z_i / Z_{i-1}.
    
    Muestrea de G_{i-1} y cuenta cuántas coloraciones son válidas para G_i.
    """
    coloring = np.random.randint(0, q, size=(K, K))
    valid_count = 0
    samples_collected = 0
    steps_executed = 0
    
    for _ in range(n_samples):
        if steps_executed + n_steps_per_sample > max_steps:
            break
        
        run_gibbs_sampler_partial(coloring, edges_i_minus_1, q, n_steps_per_sample)
        steps_executed += n_steps_per_sample
        
        if is_valid_coloring(coloring, edges_i):
            valid_count += 1
        
        samples_collected += 1
    
    ratio = valid_count / samples_collected if samples_collected > 0 else 0.0
    return ratio, samples_collected, steps_executed

In [39]:
# ============================================================================
# FUNCIÓN PRINCIPAL: CONTEO DE q-COLORACIONES
# ============================================================================

def count_colorings(K, q, n_samples, n_steps_per_sample, max_steps_per_ratio, epsilon=0.1):
    """
    Cuenta q-coloraciones en lattice K×K usando método telescópico.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    n_samples : int
        Número de muestras por ratio (usado en ejecución)
    n_steps_per_sample : int
        Pasos del Gibbs sampler por muestra (usado en ejecución)
    max_steps_per_ratio : int
        Máximo de pasos totales por ratio
    epsilon : float
        Precisión para calcular parámetros teóricos (default 0.1)
    
    Retorna:
    --------
    dict con:
        - 'K', 'q': parámetros
        - 'log_count': log(Z_{G,q})
        - 'count': Z_{G,q}
        - 'avg_ratio': ratio promedio
        - 'time': tiempo en segundos
        - 'n_samples_used': n_samples usado en ejecución
        - 'n_steps_used': n_steps_per_sample usado en ejecución
        - 'n_samples_theoretical': n_samples según Teorema 9.1
        - 'n_steps_theoretical': n_steps según Teorema 9.1
        - 'epsilon': precisión usada para cálculos teóricos
    """
    all_edges = create_lattice_edges(K)
    k = len(all_edges)  # k = 2K(K-1) aristas
    N = K * K
    
    # Z_0 = q^(K²)
    log_Z_0 = N * np.log(q)
    
    # Producto telescópico
    log_product = 0.0
    ratios = []
    
    start_time = time.time()
    
    for i in range(1, k + 1):
        edges_i_minus_1 = all_edges[:i-1] if i > 1 else np.array([], dtype=np.int64).reshape(0, 4)
        edges_i = all_edges[:i]
        
        ratio, _, _ = estimate_ratio(
            K, edges_i_minus_1, edges_i, q,
            n_samples, n_steps_per_sample, max_steps_per_ratio
        )
        
        ratio_safe = max(ratio, 1e-300)
        log_product += np.log(ratio_safe)
        ratios.append(ratio)
    
    total_time = time.time() - start_time
    
    log_count = log_Z_0 + log_product
    count = np.exp(log_count) if log_count < 700 else np.inf
    
    # Calcular parámetros teóricos (solo informativos, no afectan ejecución)
    n_samples_theo = calc_theoretical_n_samples(K, q, epsilon)
    n_steps_theo = calc_theoretical_n_steps(K, q, epsilon)
    
    return {
        'K': K,
        'q': q,
        'log_count': log_count,
        'count': count,
        'avg_ratio': np.mean(ratios),
        'time': total_time,
        # Parámetros USADOS (prácticos)
        'n_samples_used': n_samples,
        'n_steps_used': n_steps_per_sample,
        # Parámetros TEÓRICOS (Teorema 9.1)
        'n_samples_theoretical': n_samples_theo,
        'n_steps_theoretical': n_steps_theo,
        'epsilon': epsilon
    }

In [40]:
# ============================================================================
# FUNCIONES PARA PARÁMETROS TEÓRICOS (Teorema 9.1)
# ============================================================================

def calc_theoretical_n_samples(K, q, epsilon):
    """
    Calcula n_samples teórico según Teorema 9.1.
    n_samples ≤ (48 * d² * k³) / ε²
    """
    d = 4
    k = 2 * K * (K - 1)
    if k == 0:
        return 0
    return int((48 * d**2 * k**3) / (epsilon**2))


def calc_theoretical_n_steps(K, q, epsilon):
    """
    Calcula n_steps teórico según Teorema 9.1.
    n_steps ≤ k * ((2log(k) + log(ε⁻¹) + log(8)) / log(q/(q-1)) + 1)
    """
    k = 2 * K * (K - 1)
    if k == 0 or q == 1:
        return 0
    numerator = 2 * np.log(k) + np.log(1/epsilon) + np.log(8)
    denominator = np.log(q / (q - 1))
    return int(k * (numerator / denominator + 1))


# ============================================================================
# LÍMITES PRÁCTICOS - EDITA ESTOS NÚMEROS DIRECTAMENTE
# ============================================================================

MAX_SAMPLES = 1000          # Máximo de muestras permitido (para TODOS)
MAX_STEPS = 1000            # Máximo de pasos por muestra (para TODOS)
MAX_TOTAL_STEPS = 10_000_000  # Máximo de pasos totales por ratio


# ============================================================================
# FUNCIÓN PARA EJECUTAR EXPERIMENTOS
# ============================================================================

def run_experiments(K_range, q_range, output_file, epsilon=0.1):
    """
    Ejecuta experimentos para múltiples (K, q).
    
    ESTRATEGIA: Usa min(teórico, MAX) para cada parámetro.
    """
    results = []
    
    for K in K_range:
        for q in q_range:
            # Calcular teóricos
            n_samples_theo = calc_theoretical_n_samples(K, q, epsilon)
            n_steps_theo = calc_theoretical_n_steps(K, q, epsilon)
            
            # Aplicar límites: min(teórico, MAX)
            n_samples = min(n_samples_theo, MAX_SAMPLES)
            n_steps = min(n_steps_theo, MAX_STEPS)
            
            print(f"K={K:2d} q={q:2d} samples={n_samples:4d} steps={n_steps:4d} ... ", end='', flush=True)
            
            result = count_colorings(K, q, n_samples, n_steps, MAX_TOTAL_STEPS, epsilon=epsilon)
            
            print(f"Z={result['count']:10.2e}")
            
            results.append(result)
            
            # Guardar incremental
            pd.DataFrame(results).to_csv(output_file, index=False)
    
    return pd.DataFrame(results)

## Experimentos

In [42]:
# EJECUTAR experimentos completos
# K ∈ [3,20], q ∈ [2,11]
# 
# Para cambiar los límites, edita las constantes MAX_SAMPLES, MAX_STEPS arriba
# Resultados se guardan incrementalmente en CSV

df = run_experiments(
    K_range=range(3, 21),
    q_range=range(2, 12),
    output_file='../results/colorings.csv',
    epsilon=0.1
)

K= 3 q= 2 samples=1000 steps= 173 ... Z=  0.00e+00
K= 3 q= 3 samples=1000 steps= 288 ... Z= 3.79e-298
K= 3 q= 4 samples=1000 steps= 402 ... Z=  9.54e+03
K= 3 q= 5 samples=1000 steps= 514 ... Z=  1.50e+05
K= 3 q= 6 samples=1000 steps= 627 ... Z=  1.17e+06
K= 3 q= 7 samples=1000 steps= 740 ... Z=  6.18e+06
K= 3 q= 8 samples=1000 steps= 852 ... Z=  2.67e+07
K= 3 q= 9 samples=1000 steps= 964 ... Z=  9.71e+07
K= 3 q=10 samples=1000 steps=1000 ... Z=  2.70e+08
K= 3 q=11 samples=1000 steps=1000 ... Z=  7.68e+08
K= 4 q= 2 samples=1000 steps= 395 ... Z=  0.00e+00
K= 4 q= 3 samples=1000 steps= 659 ... Z=  0.00e+00
K= 4 q= 4 samples=1000 steps= 919 ... Z=  5.74e+06
K= 4 q= 5 samples=1000 steps=1000 ... Z=  8.19e+08
K= 4 q= 6 samples=1000 steps=1000 ... Z=  3.84e+10
K= 4 q= 7 samples=1000 steps=1000 ... Z=  9.08e+11
K= 4 q= 8 samples=1000 steps=1000 ... Z=  1.08e+13
K= 4 q= 9 samples=1000 steps=1000 ... Z=  1.11e+14
K= 4 q=10 samples=1000 steps=1000 ... Z=  7.22e+14
K= 4 q=11 samples=1000 steps=10

KeyboardInterrupt: 

In [ ]:
# EJECUTAR experimentos completos
# K ∈ [3,20], q ∈ [2,11]
# 
# Para cambiar los límites, edita las constantes MAX_SAMPLES, MAX_STEPS arriba
# Resultados se guardan incrementalmente en CSV

df = run_experiments(
    K_range=range(3, 21),
    q_range=range(2, 12),
    output_file='../results/colorings.csv',
    epsilon=0.1
)

K= 3 q= 2 samples=1000 steps= 173 ... Z=  0.00e+00
K= 3 q= 3 samples=1000 steps= 288 ... Z= 3.79e-298
K= 3 q= 4 samples=1000 steps= 402 ... Z=  9.54e+03
K= 3 q= 5 samples=1000 steps= 514 ... Z=  1.50e+05
K= 3 q= 6 samples=1000 steps= 627 ... Z=  1.17e+06
K= 3 q= 7 samples=1000 steps= 740 ... Z=  6.18e+06
K= 3 q= 8 samples=1000 steps= 852 ... Z=  2.67e+07
K= 3 q= 9 samples=1000 steps= 964 ... Z=  9.71e+07
K= 3 q=10 samples=1000 steps=1000 ... Z=  2.70e+08
K= 3 q=11 samples=1000 steps=1000 ... Z=  7.68e+08
K= 4 q= 2 samples=1000 steps= 395 ... Z=  0.00e+00
K= 4 q= 3 samples=1000 steps= 659 ... Z=  0.00e+00
K= 4 q= 4 samples=1000 steps= 919 ... Z=  5.74e+06
K= 4 q= 5 samples=1000 steps=1000 ... Z=  8.19e+08
K= 4 q= 6 samples=1000 steps=1000 ... Z=  3.84e+10
K= 4 q= 7 samples=1000 steps=1000 ... Z=  9.08e+11
K= 4 q= 8 samples=1000 steps=1000 ... Z=  1.08e+13
K= 4 q= 9 samples=1000 steps=1000 ... Z=  1.11e+14
K= 4 q=10 samples=1000 steps=1000 ... Z=  7.22e+14
K= 4 q=11 samples=1000 steps=10

In [ ]:
# EJECUTAR experimentos completos
# K ∈ [3,20], q ∈ [2,11]
# 
# Para cambiar los límites, edita las constantes MAX_SAMPLES, MAX_STEPS arriba
# Resultados se guardan incrementalmente en CSV

df = run_experiments(
    K_range=range(3, 21),
    q_range=range(2, 12),
    output_file='../results/colorings.csv',
    epsilon=0.1
)

K= 3 q= 2 samples=1000 steps= 173 ... Z=  0.00e+00
K= 3 q= 3 samples=1000 steps= 288 ... Z= 3.79e-298
K= 3 q= 4 samples=1000 steps= 402 ... Z=  9.54e+03
K= 3 q= 5 samples=1000 steps= 514 ... Z=  1.50e+05
K= 3 q= 6 samples=1000 steps= 627 ... Z=  1.17e+06
K= 3 q= 7 samples=1000 steps= 740 ... Z=  6.18e+06
K= 3 q= 8 samples=1000 steps= 852 ... Z=  2.67e+07
K= 3 q= 9 samples=1000 steps= 964 ... Z=  9.71e+07
K= 3 q=10 samples=1000 steps=1000 ... Z=  2.70e+08
K= 3 q=11 samples=1000 steps=1000 ... Z=  7.68e+08
K= 4 q= 2 samples=1000 steps= 395 ... Z=  0.00e+00
K= 4 q= 3 samples=1000 steps= 659 ... Z=  0.00e+00
K= 4 q= 4 samples=1000 steps= 919 ... Z=  5.74e+06
K= 4 q= 5 samples=1000 steps=1000 ... Z=  8.19e+08
K= 4 q= 6 samples=1000 steps=1000 ... Z=  3.84e+10
K= 4 q= 7 samples=1000 steps=1000 ... Z=  9.08e+11
K= 4 q= 8 samples=1000 steps=1000 ... Z=  1.08e+13
K= 4 q= 9 samples=1000 steps=1000 ... Z=  1.11e+14
K= 4 q=10 samples=1000 steps=1000 ... Z=  7.22e+14
K= 4 q=11 samples=1000 steps=10